In [1]:
import numpy as np
import cvxpy as cvx

In [118]:
def stationary_dist(transitions, input_given_state):
    m, n = policy.shape
    
    P = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            P[i, j] = np.sum(transitions[i, j, :] * policy[:, i])

    w, v = np.linalg.eig(P.transpose())
    stationary = v[:, w.argmax()]
    stationary =  stationary / stationary.sum()

def value_iter(costs, transitions, init_dist, discount=0.99, iters=100, verbose=False):
    n, m = costs.shape
    policy = np.zeros((m, n))
    V = np.zeros(n)

    for iter in range(iters):
        transition_vals = transitions.sum(axis=1) * costs + discount * (transitions * V.reshape((1, n, 1))).sum(axis=1)

        policy = np.zeros((m, n))
        policy[transition_vals.argmin(axis=1), np.arange(n)] = 1
        V = transition_vals.min(axis=1)

        if verbose:
            print(f'[{iter}] Obj Val: {V @ init_dist}')
    
    return V @ init_dist, V, policy

def value_func(costs, transitions, policy, discount=0.99):
    n, m = costs.shape
    
    P = np.zeros((n, n))
    R = np.zeros(n)
    
    for i in range(n):
        for j in range(n):
            P[i, j] = np.sum(transitions[i, j, :] * policy[:, i])
        R[i] = costs[i, :] @ policy[:, i]
        
    A = discount * P - np.eye(n)
    b = -P.sum(axis=1) * R
    
    V = np.linalg.solve(A, b)
    
    return V

def diff_priv_iter(costs, transitions, init_dist, privacy, discount=0.99, iters=100, verbose=False):
    n, m = costs.shape

    obj = np.zeros(iters)
    
    stationary = cvx.Parameter(n)
    V_prev = cvx.Parameter(n, value=value_func(costs, transitions, policy, discount))
    
    policy = cvx.Variable((n, m))
    
    for iter in range(iters):
        #V.value = value_func(costs, transitions, policy, iters, verbose)
        stationary.value = stationary_dist(transitions, policy)
        
        V_next = 
        
        cstr = [policy >= 0, cvx.sum(policy, 0) == 1]
        
        
        

In [57]:
n = 5
m = 3

goal = 2

transitions = np.zeros((n, n, m))
costs = np.ones((n, m))
init_dist = np.array([0.25, 0.25, 0.25, 0.25, 0])


transitions[0, 0, 0:1] = 1
transitions[0, 1, 2] = 1

for i in range(1, n - 1):
    transitions[i, i - 1, 0] = 1
    transitions[i, i, 1] = 1
    transitions[i, i + 1, 2] = 1

transitions[n - 1, n - 1, :] = 1

costs[goal - 1, 2] = -1
costs[goal, 1] = -1
costs[goal + 1, 0] = -1


In [58]:
costs

array([[ 1.,  1.,  1.],
       [ 1.,  1., -1.],
       [ 1., -1.,  1.],
       [-1.,  1.,  1.],
       [ 1.,  1.,  1.]])

In [125]:
obj, V, policy = value_iter(costs, transitions, init_dist, iters=10000)

In [126]:
V

array([ -98., -100., -100., -100.,  100.])

In [127]:
value_func(costs, transitions, policy)

array([ -98., -100., -100., -100.,  100.])

In [101]:
v[:, 4]

array([0., 0., 0., 0., 1.])